In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

import keras
import keras.backend
from keras import layers
from keras import models
from keras import optimizers
import keras.utils

print(tf.__version__)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sklearn
from sklearn.utils import shuffle

import os
import pickle

/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages

1.13.1


## Useful Functions

In [15]:
def build_CNN():
    input2 = [layers.Input(shape = (len(X_train[i][0]),)) for i in range(1,len(X_train))]
    input1 = layers.Input(shape = (grid, grid,1))
    x = layers.Conv2D(32, (5, 5), activation = 'relu', padding = 'same')(input1)
    x = layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(x)
    x = layers.Conv2D(32, (2, 2), activation = 'relu', padding = 'same')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.MaxPool2D((2, 2))(x)
    x1 = layers.Flatten()(x)
    if input2 == []:
        x = layers.Dense(64, activation='relu', name = 'dense1')(x1)
    else:
        x = layers.concatenate(inputs = [x1] + input2, axis = -1)
        x = layers.Dense(64, activation='relu', name = 'dense2')(x)
    x = layers.Dense(128, activation='relu', name = 'dense3')(x)
    output = layers.Dense(2, activation='softmax')(x)
    model = models.Model(inputs= [input1] + input2,
                         outputs = output)
    opt = keras.optimizers.Adam(lr = 0.001,
                                beta_1 = 0.9,
                                beta_2 = 0.999,
                                amsgrad = False)
    model.compile(loss = 'binary_crossentropy',
                optimizer = opt,
                metrics = ['binary_crossentropy', 'accuracy'])
    return model

In [3]:
def build_DNN():
    input2 = [layers.Input(shape = (len(X_train[i][0]),)) for i in range(0,len(X_train))]
    if len(input2) == 1:
        x = layers.Dense(64,activation='relu')(input2[0])
    else:
        x = layers.concatenate(inputs = input2, axis = -1)
        x = layers.Dense(64, activation = 'relu')(x)
    x = layers.Dense(128, activation = 'relu')(x)
    output = layers.Dense(2, activation = 'softmax')(x)
    model = models.Model(inputs = input2, 
                         outputs = output)
    opt=keras.optimizers.Adam(lr = 0.0005,
                              beta_1 = 0.9,
                              beta_2 = 0.9,
                              amsgrad = False)
    model.compile(loss = 'binary_crossentropy',
                optimizer = opt,
                metrics = ['binary_crossentropy', 'accuracy'])
    return model

In [4]:
def build_XY(features,label,dic):
    X = [dic[key] for key in features]
    Y = [dic[key] for key in label]
    dim = [ele.shape+(1,) for ele in X]
    for i in range(0,len(features)):
        X[i] = X[i].reshape(dim[i])
    return X,Y

## Import data:
Using Train and Test set produced by preprocessing notebook.

In [5]:
grid = 16
data_train = np.load('../data/ShowJet_final_train.npz')

In [6]:
n_train = len(data_train['jetPt'])
print(n_train)

1677864


In [7]:
removeFeat = ['tau1_b15', 'tau2_b15', 'tau3_b15',
            'tau1_sd_b15', 'tau2_sd_b15', 'tau3_sd_b15',
            'labels']
feat_all = [key for key in data_train.keys()];
for feat in removeFeat:
    feat_all.remove(feat)
feat_all

['jetImages',
 'jetPt',
 'jetEta',
 'jetPhi',
 'jetMass',
 'jetMassSD',
 'tau1_b05',
 'tau2_b05',
 'tau3_b05',
 'tau1_sd_b05',
 'tau2_sd_b05',
 'tau3_sd_b05',
 'tau1_b10',
 'tau2_b10',
 'tau3_b10',
 'tau1_sd_b10',
 'tau2_sd_b10',
 'tau3_sd_b10',
 'tau1_b20',
 'tau2_b20',
 'tau3_b20',
 'tau1_sd_b20',
 'tau2_sd_b20',
 'tau3_sd_b20',
 'chMult',
 'neutMult',
 'phoMult',
 'eleMult',
 'muMult',
 'jetpull',
 'beta3',
 'beta3_sd',
 'tau21']

# Build CNN

## Build Model with only Images

In [8]:
# features will be loaded into X
features = ['jetImages']

# label into Y
label = ['labels']

In [10]:
X_train, Y_train = build_XY(features,label,data_train)

In [11]:
CNN = build_CNN()

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

10/10 [==============================] - 0s 4ms/step
[[0.5001186  0.4998814 ]
 [0.50022477 0.49977526]
 [0.5001654  0.49983454]
 [0.5001983  0.49980167]
 [0.5003264  0.49967366]
 [0.500041   0.49995905]
 [0.5004841  0.49951592]
 [0.5007872  0.49921286]
 [0.50026095 0.49973908]
 [0.49996716 0.5000329 ]]
[0.6934884190559387, 0.6934884190559387, 0.30000001192092896]


In [13]:
checkpoint_path = "model/CNN_sub_0.h2"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 60
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS, 
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

Instructions for updating:
Use tf.cast instead.
Train on 1342291 samples, validate on 335573 samples
Epoch 1/60
1342291/1342291 [==============================] - 1867s 1ms/step - loss: 0.5608 - binary_crossentropy: 0.5608 - acc: 0.7102 - val_loss: 0.5421 - val_binary_crossentropy: 0.5421 - val_acc: 0.7237

Epoch 00001: val_loss improved from inf to 0.54208, saving model to model/CNN_sub_0.h2
Epoch 2/60
1342291/1342291 [==============================] - 1726s 1ms/step - loss: 0.5397 - binary_crossentropy: 0.5397 - acc: 0.7263 - val_loss: 0.5347 - val_binary_crossentropy: 0.5347 - val_acc: 0.7289

Epoch 00002: val_loss improved from 0.54208 to 0.53468, saving model to model/CNN_sub_0.h2
Epoch 3/60
1342291/1342291 [==============================] - 2254s 2ms/step - loss: 0.5324 - binary_crossentropy: 0.5324 - acc: 0.7315 - val_loss: 0.5347 - val_binary_crossentropy: 0.5347 - val_acc: 0.7302

Epoch 00003: val_loss did not improve from 0.53468
Epoch 4/60
1342291/1342291 [==================

In [16]:
with open('history/CNN_sub_0.h2.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

## Build Model with all Features

### Select Features for Model

Show all possible features

In [16]:
# features will be loaded into X
features = feat_all

# label into Y
label = ['labels']

In [17]:
X_train, Y_train = build_XY(features,label,data_train)

Initialize model:

In [22]:
CNN = build_CNN()

Test model's prediction $before$ training:

In [23]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

10/10 [==============================] - 0s 8ms/step
[[0.45223165 0.54776835]
 [0.46858796 0.53141207]
 [0.47678593 0.52321404]
 [0.48226115 0.5177388 ]
 [0.47568065 0.52431935]
 [0.46966124 0.53033876]
 [0.4805692  0.51943076]
 [0.46108103 0.5389189 ]
 [0.4826258  0.51737416]
 [0.47243115 0.5275688 ]]
[0.6786078810691833, 0.6786078810691833, 0.6000000238418579]


Now train! (warning: if building CNN, computer tends to get loud)

In [24]:
checkpoint_path = "model/CNN_all.h8"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 60
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

Train on 1342291 samples, validate on 335573 samples
Epoch 1/60
1342291/1342291 [==============================] - 1580s 1ms/step - loss: 0.3542 - binary_crossentropy: 0.3542 - acc: 0.8446 - val_loss: 0.3314 - val_binary_crossentropy: 0.3314 - val_acc: 0.8557

Epoch 00001: val_loss improved from inf to 0.33144, saving model to model/CNN_all.h8
Epoch 2/60
1342291/1342291 [==============================] - 1778s 1ms/step - loss: 0.3320 - binary_crossentropy: 0.3320 - acc: 0.8557 - val_loss: 0.3233 - val_binary_crossentropy: 0.3233 - val_acc: 0.8604

Epoch 00002: val_loss improved from 0.33144 to 0.32329, saving model to model/CNN_all.h8
Epoch 3/60
1342291/1342291 [==============================] - 1719s 1ms/step - loss: 0.3265 - binary_crossentropy: 0.3265 - acc: 0.8583 - val_loss: 0.3290 - val_binary_crossentropy: 0.3290 - val_acc: 0.8565

Epoch 00003: val_loss did not improve from 0.32329
Epoch 4/60
1342291/1342291 [==============================] - 1842s 1ms/step - loss: 0.3231 - bina


Epoch 00030: val_loss did not improve from 0.30763
Epoch 31/60
1342291/1342291 [==============================] - 799s 595us/step - loss: 0.3089 - binary_crossentropy: 0.3089 - acc: 0.8670 - val_loss: 0.3072 - val_binary_crossentropy: 0.3072 - val_acc: 0.8673

Epoch 00031: val_loss improved from 0.30763 to 0.30722, saving model to model/CNN_all.h8
Epoch 32/60
1342291/1342291 [==============================] - 798s 595us/step - loss: 0.3085 - binary_crossentropy: 0.3085 - acc: 0.8671 - val_loss: 0.3076 - val_binary_crossentropy: 0.3076 - val_acc: 0.8671

Epoch 00032: val_loss did not improve from 0.30722
Epoch 33/60
1342291/1342291 [==============================] - 799s 595us/step - loss: 0.3084 - binary_crossentropy: 0.3084 - acc: 0.8671 - val_loss: 0.3081 - val_binary_crossentropy: 0.3081 - val_acc: 0.8679

Epoch 00033: val_loss did not improve from 0.30722
Epoch 34/60
1342291/1342291 [==============================] - 799s 595us/step - loss: 0.3083 - binary_crossentropy: 0.3083 - a

In [25]:
with open('history/CNN_all.h8.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

### Select Features for Model

Show all possible features

In [30]:
feat_all

['jetImages',
 'jetPt',
 'jetEta',
 'jetPhi',
 'jetMass',
 'jetMassSD',
 'tau1_b05',
 'tau2_b05',
 'tau3_b05',
 'tau1_sd_b05',
 'tau2_sd_b05',
 'tau3_sd_b05',
 'tau1_b10',
 'tau2_b10',
 'tau3_b10',
 'tau1_sd_b10',
 'tau2_sd_b10',
 'tau3_sd_b10',
 'tau1_b20',
 'tau2_b20',
 'tau3_b20',
 'tau1_sd_b20',
 'tau2_sd_b20',
 'tau3_sd_b20',
 'chMult',
 'neutMult',
 'phoMult',
 'eleMult',
 'muMult',
 'jetpull',
 'beta3',
 'beta3_sd',
 'tau21']

In [26]:
drop_those = [ 'jetImages',
 'chMult',
 'neutMult',
 'phoMult',
 'eleMult',
 'muMult']
# features will be loaded into X
features = feat_all[:]
for feat in drop_those:
    features.remove(feat)
# label into Y
label = ['labels']

In [27]:
features

['jetPt',
 'jetEta',
 'jetPhi',
 'jetMass',
 'jetMassSD',
 'tau1_b05',
 'tau2_b05',
 'tau3_b05',
 'tau1_sd_b05',
 'tau2_sd_b05',
 'tau3_sd_b05',
 'tau1_b10',
 'tau2_b10',
 'tau3_b10',
 'tau1_sd_b10',
 'tau2_sd_b10',
 'tau3_sd_b10',
 'tau1_b20',
 'tau2_b20',
 'tau3_b20',
 'tau1_sd_b20',
 'tau2_sd_b20',
 'tau3_sd_b20',
 'jetpull',
 'beta3',
 'beta3_sd',
 'tau21']

In [28]:
X_train, Y_train = build_XY(features,label,data_train)

Initialize model:

In [29]:
CNN = build_DNN()

Test model's prediction $before$ training:

In [30]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

10/10 [==============================] - 0s 10ms/step
[[0.5270552  0.4729448 ]
 [0.5053764  0.4946237 ]
 [0.5052645  0.49473548]
 [0.51708007 0.48291987]
 [0.48523638 0.51476365]
 [0.54981667 0.45018327]
 [0.5066523  0.4933477 ]
 [0.5062908  0.49370918]
 [0.51445496 0.48554504]
 [0.5148447  0.48515528]]
[0.6866416931152344, 0.6866416931152344, 0.5]


Now train! (warning: if building CNN, computer tends to get loud)

In [31]:
checkpoint_path = "model/DNN_noflavour.h1"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 60
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

Train on 1342291 samples, validate on 335573 samples
Epoch 1/60
1342291/1342291 [==============================] - 78s 58us/step - loss: 0.3736 - binary_crossentropy: 0.3736 - acc: 0.8358 - val_loss: 0.3517 - val_binary_crossentropy: 0.3517 - val_acc: 0.8485

Epoch 00001: val_loss improved from inf to 0.35172, saving model to model/DNN_noflavour.h1
Epoch 2/60
1342291/1342291 [==============================] - 81s 60us/step - loss: 0.3526 - binary_crossentropy: 0.3526 - acc: 0.8483 - val_loss: 0.3554 - val_binary_crossentropy: 0.3554 - val_acc: 0.8486

Epoch 00002: val_loss did not improve from 0.35172
Epoch 3/60
1342291/1342291 [==============================] - 83s 62us/step - loss: 0.3563 - binary_crossentropy: 0.3563 - acc: 0.8490 - val_loss: 0.3508 - val_binary_crossentropy: 0.3508 - val_acc: 0.8504

Epoch 00003: val_loss improved from 0.35172 to 0.35081, saving model to model/DNN_noflavour.h1
Epoch 4/60
1342291/1342291 [==============================] - 83s 62us/step - loss: 0.357

In [32]:
with open('history/DNN_noflavour.h1.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)